In [ ]:
import os.path
from llama_index.core import (
    VectorStoreIndex,
    SimpleDirectoryReader,
    StorageContext,
    load_index_from_storage,
)

import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

### Create and fill vector storage

In [ ]:
# check if storage already exists
PERSIST_DIR = "./storage"
if not os.path.exists(PERSIST_DIR):
    # load the documents and create the index
    documents = SimpleDirectoryReader("data").load_data()
    index = VectorStoreIndex.from_documents(documents)
    # store it for later
    index.storage_context.persist(persist_dir=PERSIST_DIR)
else:
    # load the existing index
    storage_context = StorageContext.from_defaults(persist_dir=PERSIST_DIR)
    index = load_index_from_storage(storage_context)

### Query openAIs API

In [ ]:

query_engine = index.as_query_engine()

query = "What did the author do growing up? Give an extremely short answer."
query = "Wer ist der Author?"
query = "Von wann ist der Text?"
query = "Welche Farbe hat das Eichhörnchen?"

response = query_engine.query(query)
print(response)

## Run local model, retrieved by Ollama

In [ ]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader, Settings
from llama_index.core.embeddings import resolve_embed_model
from llama_index.llms.ollama import Ollama

documents = SimpleDirectoryReader("data").load_data()

# # bge-m3 embedding model
# Settings.embed_model = resolve_embed_model("local:BAAI/bge-small-en-v1.5")

# # ollama
# Settings.llm = Ollama(model="mistral", request_timeout=30.0)

# index = VectorStoreIndex.from_documents(
#     documents,
# )